# Dependencies and Setup

In [12]:
import pandas as pd

In [13]:
drug_related_deaths_df = pd.read_csv('../Datasets//Accidental_Drug_Related_Deaths_2012-2018.csv')

In [14]:
narrowed_df = drug_related_deaths_df.loc[:,['Date','Age','Sex','Race','DeathCity','DeathCityGeo','COD','Heroin','Cocaine','Fentanyl','FentanylAnalogue','Oxycodone','Oxymorphone','Ethanol','Hydrocodone','Benzodiazepine','Methadone','Amphet','Tramad','Morphine_NotHeroin','Hydromorphone','Other','OpiateNOS','AnyOpioid']]

In [15]:
# Drop records without Age, Sex and Race entry
narrowed_df = narrowed_df.dropna(subset=['Age', 'Sex','Race'], how='all')

In [16]:
narrowed_df.update(narrowed_df[['Heroin','Cocaine','Fentanyl','FentanylAnalogue','Oxycodone','Oxymorphone','Ethanol','Hydrocodone','Benzodiazepine','Methadone','Amphet','Tramad','Morphine_NotHeroin','Hydromorphone','Other','OpiateNOS','AnyOpioid']].fillna('N'))
narrowed_df['Date'] = pd.to_datetime(narrowed_df['Date']).dt.strftime('%m/%d/%Y')
narrowed_df

,Date,Age,Sex,Race,DeathCity,DeathCityGeo,COD,Heroin,Cocaine,Fentanyl,...,Hydrocodone,Benzodiazepine,Methadone,Amphet,Tramad,Morphine_NotHeroin,Hydromorphone,Other,OpiateNOS,AnyOpioid
1,03/21/2013,48.0,Male,Black,NORWALK,"Norwalk, CT\n(41.11805, -73.412906)",Cocaine Intoxication,N,Y,N,...,N,N,N,N,N,N,N,N,N,N
2,03/13/2016,30.0,Female,White,DANBURY,"Danbury, CT\n(41.393666, -73.451539)",Acute Heroin and Cocaine Intoxication,Y,Y,N,...,N,N,N,N,N,N,N,N,N,Y
3,03/31/2016,23.0,Male,White,GREENWICH,"Greenwich, CT\n(41.026526, -73.628549)",Acute Fentanyl and Morphine Intoxication,Y,N,Y,...,N,N,N,N,N,N,N,N,N,Y
4,02/13/2013,22.0,Male,"Asian, Other",GREENWICH,"Greenwich, CT\n(41.026526, -73.628549)",Fentanyl Intoxication,N,N,Y,...,N,N,N,N,N,N,N,N,N,N
5,06/29/2014,23.0,Male,White,BRISTOL,"BRISTOL, CT\n(41.673037, -72.945791)",Heroin Intoxication,Y,N,N,...,N,N,N,N,N,N,N,N,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5100,09/08/2015,43.0,Male,White,CHESHIRE,"CHESHIRE, CT\n(41.498834, -72.901448)",Acute Intoxication due to the Combined Effects...,N,N,N,...,N,Y,N,N,N,N,N,N,N,N
5101,07/22/2017,21.0,Male,White,NEW HAVEN,"New Haven, CT\n(41.308252, -72.924161)","Multidrug Toxicity Including Heroin, Alprazola...",Y,N,N,...,N,Y,N,N,N,N,N,N,N,N
5102,08/14/2018,30.0,Male,White,DANBURY,"DANBURY, CT\n(41.393666, -73.451539)","Multidrug Toxicity Including Heroin, Fentanyl,...",Y,N,Y,...,N,N,N,N,Y,N,N,N,N,Y
5103,03/16/2014,33.0,Male,White,WINDSOR,"WINDSOR, CT\n(41.852781, -72.64379)",Acute Fentanyl intoxication,N,N,Y,...,N,N,N,N,N,N,N,N,N,N
